In [22]:
# Dependencies and Setup #see VacationPY for references
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key
import json
import matplotlib.pyplot as plt
from config import weather_api_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Poum,-20.2333,164.0167,72.61,66,75,6.96,NC
1,1,Punta Arenas,-53.1500,-70.9167,44.71,53,40,23.02,CL
2,2,Beloha,-25.1667,45.0500,68.18,79,0,5.91,MG
3,3,Troitskoye,52.9821,84.6757,41.00,72,100,13.89,RU
4,4,Eastbourne,50.7687,0.2845,60.17,91,100,4.12,GB


In [26]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)                      

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,Poum,-20.2333,164.0167,72.61,66,75,6.96,NC
8,8,Yabassi,4.4570,9.9682,74.84,98,100,2.26,CM
9,9,Naze,28.3667,129.4833,77.40,86,44,10.42,JP
13,13,Oranjestad,12.5240,-70.0270,87.75,75,20,11.01,AW
14,14,Hamilton,39.1834,-84.5333,71.04,46,1,1.99,US
17,17,Rikitea,-23.1203,-134.9692,73.90,76,58,14.74,PF
21,21,Canyon,34.9803,-101.9188,91.47,18,23,15.61,US
22,22,Semnan,35.5729,53.3971,74.19,15,0,3.85,IR
23,23,Chanute,37.6792,-95.4572,81.91,32,1,4.61,US
25,25,Cumberland,35.0502,-78.8664,80.08,40,1,8.05,US


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       1
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset=['Country'], inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [30]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
dtype: int64

In [31]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pd.DataFrame(preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy())
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Poum,NC,72.61,-20.2333,164.0167,
8,Yabassi,CM,74.84,4.4570,9.9682,
9,Naze,JP,77.40,28.3667,129.4833,
13,Oranjestad,AW,87.75,12.5240,-70.0270,
14,Hamilton,US,71.04,39.1834,-84.5333,
17,Rikitea,PF,73.90,-23.1203,-134.9692,
21,Canyon,US,91.47,34.9803,-101.9188,
22,Semnan,IR,74.19,35.5729,53.3971,
23,Chanute,US,81.91,37.6792,-95.4572,
25,Cumberland,US,80.08,35.0502,-78.8664,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [33]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
 
    # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [34]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Poum,NC,72.61,-20.2333,164.0167,
8,Yabassi,CM,74.84,4.4570,9.9682,
9,Naze,JP,77.40,28.3667,129.4833,
13,Oranjestad,AW,87.75,12.5240,-70.0270,
14,Hamilton,US,71.04,39.1834,-84.5333,
...,...,...,...,...,...,...
568,Batticaloa,LK,80.28,7.7102,81.6924,
569,Bodden Town,KY,88.99,19.2833,-81.2500,
571,Mandan,US,85.03,46.8267,-100.8896,
573,Eldorado,BR,85.08,-23.7869,-54.2836,


In [35]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df   

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Poum,NC,72.61,-20.2333,164.0167,NaN
8,Yabassi,CM,74.84,4.4570,9.9682,NaN
9,Naze,JP,77.40,28.3667,129.4833,NaN
13,Oranjestad,AW,87.75,12.5240,-70.0270,NaN
14,Hamilton,US,71.04,39.1834,-84.5333,NaN
...,...,...,...,...,...,...
568,Batticaloa,LK,80.28,7.7102,81.6924,NaN
569,Bodden Town,KY,88.99,19.2833,-81.2500,NaN
571,Mandan,US,85.03,46.8267,-100.8896,NaN
573,Eldorado,BR,85.08,-23.7869,-54.2836,NaN


In [36]:
hotel_df.isnull().sum()

City            0
Country         0
Max Temp        0
Lat             0
Lng             0
Hotel Name    279
dtype: int64

In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = pd.DataFrame(hotel_df.dropna(subset=["Hotel Name"], inplace=True))
clean_hotel_df

""


In [38]:
# 8a. Create the output File (CSV)
output_data_file = "weather_data/hotels.csv"
clean_hotel_df.to_csv(output_data_file, index_label="Hotel Name")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""


In [43]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [44]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [49]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

hotel_info

[]

In [50]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [47]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))